In [88]:
#11. Hvor stor andel av personbilene bilene som selges i Norge har firehjulstrekk?
file_path = "kjoretoyinfo_preppet.parquet"
df = pd.read_parquet(file_path)

In [89]:
relevant_kolonner = ["tekn_aksler_drift"]

In [90]:
personbiler_df = df[df["tekn_modell"] == "Personbil"]

In [91]:
filtered_df = personbiler_df[relevant_kolonner]

In [92]:
firehjulstrekk_andel = (filtered_df["tekn_aksler_drift"] == "To aksler er firehjulsdrift").mean() * 100

In [93]:
print(f"Andelen personbiler med firehjulstrekk er: {firehjulstrekk_andel}%"
    if pd.notna(firehjulstrekk_andel) 
        else "Ingen personbiler med informasjon om firehjulsdrift funnet.")

Ingen personbiler med informasjon om firehjulsdrift funnet.


In [94]:
#12. Hvilken måned i året førstegangsregistreres det flest biler i Norge?
file_path = "kjoretoyinfo_preppet.parquet"
df = pd.read_parquet(file_path, engine='pyarrow')

In [95]:
df['tekn_reg_f_g_n'] = pd.to_datetime(df['tekn_reg_f_g_n'], errors='coerce')

In [96]:
mest_populær_måned = df['tekn_reg_f_g_n'].dt.month.value_counts().idxmax()

In [97]:
print(f"Måneden med flest førstegangsregistreringer er: {mest_populær_måned}")

Måneden med flest førstegangsregistreringer er: 12
